In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset/movies_metadata_ready_big.csv', encoding='utf-8')
df = df.dropna()
df.shape

C:\Users\wsnia\AppData\Local\Temp\ipykernel_6300\1046244740.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/movies_metadata_ready_big.csv', encoding='utf-8')


(20390, 16)

In [7]:
df.columns

Index(['adult', 'id', 'original_language', 'original_title', 'overview',
       'popularity', 'runtime', 'tagline', 'title', 'vote_average',
       'vote_count', 'genres', 'production_companies', 'production_countries',
       'release_year', 'overview_keywords'],
      dtype='object')

In [10]:
df.sort_values(by='vote_count', ascending=False).head(20)

,adult,id,original_language,original_title,overview,popularity,runtime,tagline,title,vote_average,vote_count,genres,production_companies,production_countries,release_year,overview_keywords
15480,False,27205,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.108149,148.0,Your mind is the scene of the crime.,Inception,8.1,14075.0,"['Action', 'Thriller', 'Science Fiction', 'Mys...",[],[],2010.0,"['cobb', 'skilled', 'thief', 'commit', 'corpor..."
12481,False,155,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167259,152.0,Why So Serious?,The Dark Knight,8.3,12269.0,"['Drama', 'Action', 'Crime', 'Thriller']",[],[],2008.0,"['batman', 'raise', 'stake', 'war', 'crime', '..."
14551,False,19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.070892,162.0,Enter the World of Pandora.,Avatar,7.2,12114.0,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",[],[],2009.0,"['nd', 'century', 'paraplegic', 'marine', 'dis..."
17818,False,24428,en,The Avengers,When an unexpected enemy emerges and threatens...,89.887648,143.0,Some assembly required.,The Avengers,7.4,12000.0,"['Science Fiction', 'Action', 'Adventure']",[],[],2012.0,"['unexpected', 'enemy', 'emerge', 'threaten', ..."
26564,False,293660,en,Deadpool,Deadpool tells the origin story of former Spec...,187.860492,108.0,Witness the beginning of a happy ending,Deadpool,7.4,11444.0,"['Action', 'Adventure', 'Comedy']",[],[],2016.0,"['deadpool', 'tell', 'origin', 'story', 'speci..."
22879,False,157336,en,Interstellar,Interstellar chronicles the adventures of a gr...,32.213481,169.0,Mankind was born on Earth. It was never meant ...,Interstellar,8.1,11187.0,"['Adventure', 'Drama', 'Science Fiction']",[],[],2014.0,"['interstellar', 'chronicle', 'adventure', 'gr..."
20051,False,68718,en,Django Unchained,"With the help of a German bounty hunter, a fre...",19.785025,165.0,"Life, liberty and the pursuit of vengeance.",Django Unchained,7.8,10297.0,"['Drama', 'Western']",[],[],2012.0,"['help', 'german', 'bounty', 'hunter', 'freed'..."
23753,False,118340,en,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",53.291601,121.0,All heroes start somewhere.,Guardians of the Galaxy,7.9,10014.0,"['Action', 'Science Fiction', 'Adventure']",[],[],2014.0,"['light', 'year', 'earth', 'year', 'abduct', '..."
2843,False,550,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,63.869599,139.0,Mischief. Mayhem. Soap.,Fight Club,8.3,9678.0,['Drama'],[],[],1999.0,"['tickingtimebomb', 'insomniac', 'slippery', '..."
18244,False,70160,en,The Hunger Games,Every year in the ruins of what was once North...,20.031667,142.0,May The Odds Be Ever In Your Favor.,The Hunger Games,6.9,9634.0,"['Science Fiction', 'Adventure', 'Fantasy']",[],[],2012.0,"['year', 'ruin', 'north', 'america', 'nation',..."


In [16]:
def compare_multi_field(f1, f2):
    try:
        return len(set(f1).intersection(set(f2))) / len(set(f1).union(set(f2)))
    except:
        return 0

def the_most_similar_movies(movie_id, n=5, w_adult = 0.1, w_original_language = 0.1, w_release_year = 0.1, w_genres = 0.3, w_production_companies = 0.1, w_production_countries = 0.1, w_overview_keyword = 0.3, w_popularity = 0.1):
    movie_1 = df.loc[movie_id]
    print(movie_1["title"], movie_1["genres"])
    similarities = []
    for i, movie_2 in df.iterrows():
        if i == movie_id:
            continue
        adult = 1 if movie_1['adult'] == movie_2['adult'] else 0
        original_language = 1 if movie_1['original_language'] == movie_2['original_language'] else 0
        release_year = 1 - (abs(movie_1['release_year'] - movie_2['release_year']) / 100)
        genres = compare_multi_field(movie_1['genres'], movie_2['genres'])
        production_companies = compare_multi_field(movie_1['production_companies'], movie_2['production_companies'])
        production_countries = compare_multi_field(movie_1['production_countries'], movie_2['production_countries'])
        overview_keywords = compare_multi_field(movie_1['overview_keywords'], movie_2['overview_keywords'])
        try:
            popularity = 1 - (abs(float(movie_1['popularity']) - float(movie_2['popularity'])) / 100)
        except:
            print(movie_1['popularity'], movie_2['popularity'], "!!!!!!!!!!!!!!!")
            popularity = 0
        # popularity = 1 - (abs(movie_1['popularity'] - movie_2['popularity']) / 100)
        similarity = w_adult * adult +  w_original_language * original_language + w_release_year * release_year + w_genres * genres + w_production_companies * production_companies + w_production_countries * production_countries + w_overview_keyword * overview_keywords + w_popularity * popularity
        similarities.append((movie_2["title"], similarity.item(), movie_2["genres"]))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return similarities[:n]

In [19]:
the_most_similar_movies(12588, 5)

Iron Man ['Action', 'Science Fiction', 'Adventure']


[('Captain America: The First Avenger',
  1.183905655413793,
  "['Action', 'Adventure', 'Science Fiction']"),
 ('The Hunger Games: Catching Fire',
  1.1814191324137933,
  "['Adventure', 'Action', 'Science Fiction']"),
 ('Star Trek Into Darkness',
  1.178363363413793,
  "['Action', 'Adventure', 'Science Fiction']"),
 ('The Incredible Hulk',
  1.177052438,
  "['Science Fiction', 'Action', 'Adventure']"),
 ('TRON: Legacy',
  1.1764962458275863,
  "['Adventure', 'Action', 'Science Fiction']")]

In [20]:
the_most_similar_movies(314, 10)

The Shawshank Redemption ['Drama', 'Crime']


[('The Godfather', 1.1467742058275863, "['Drama', 'Crime']"),
 ('The Godfather: Part II', 1.1442942488275862, "['Drama', 'Crime']"),
 ('A Guide To Recognizing Your Saints',
  1.1437030570000002,
  "['Crime', 'Drama']"),
 ('Bad Lieutenant', 1.1420573482857141, "['Crime', 'Drama']"),
 ('Mean Creek', 1.1404380384137933, "['Crime', 'Drama']"),
 ('Catch Me If You Can', 1.1394980188275863, "['Drama', 'Crime']"),
 ('Bad Boy Bubby', 1.137186145827586, "['Crime', 'Drama']"),
 ('Green Street Hooligans', 1.134782387413793, "['Crime', 'Drama']"),
 ('Blow', 1.132997164827586, "['Crime', 'Drama']"),
 ('A Bronx Tale', 1.1327805698275861, "['Drama', 'Crime']")]

In [24]:
the_most_similar_movies(834, 15)

The Godfather ['Drama', 'Crime']


[('The Godfather: Part II', 1.1728303878275863, "['Drama', 'Crime']"),
 ('Taxi Driver', 1.1586386214137931, "['Crime', 'Drama']"),
 ('Badlands', 1.1531041405714284, "['Crime', 'Drama']"),
 ('The Outfit', 1.149287534285714, "['Crime', 'Drama']"),
 ('On the Waterfront', 1.1487570014137933, "['Crime', 'Drama']"),
 ('The Gambler', 1.1479511754137932, "['Drama', 'Crime']"),
 ('The Shawshank Redemption', 1.1467742058275863, "['Drama', 'Crime']"),
 ('Report to the Commissioner', 1.1456919942857142, "['Crime', 'Drama']"),
 ('Scum', 1.1445899132857142, "['Crime', 'Drama']"),
 ('Made in Britain', 1.140274327285714, "['Crime', 'Drama']"),
 ('The Ceremony', 1.140084846285714, "['Drama', 'Crime']"),
 ('Midnight Express', 1.1393871288275863, "['Drama', 'Crime']"),
 ('House of Women', 1.1381783342857141, "['Crime', 'Drama']"),
 ('Payroll', 1.1379362172857144, "['Crime', 'Drama']"),
 ('Once Upon a Time in America', 1.1376942766551723, "['Drama', 'Crime']")]